In [56]:
#pip install docx2txt

In [57]:
import docx2txt
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
from nltk.corpus import stopwords
import re
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [58]:
import pandas as pd
from nltk.corpus import stopwords
import re

# load data
#df = pd.read_csv('./jobs_small.csv', encoding="latin-1")
df = pd.read_excel('./data/jd_clean.xlsx')
df_course= pd.read_excel('./data/course_clean.xlsx')

### Input your course code

In [59]:
#mycourse=['ISSS622','ISSS602','ISSS616','ISSS621']
#mycourse=['CS602','CS601','CS610','CS606']
mycourse=['ISFS614','ISFS621','ISFS620','ISSS622']

In [60]:
df_data=df.dropna(subset=['Title','FullDescription','Company'])
#df.info()

### Create a corpus of Job description and Course introduction

In [66]:
for i in df_data.index:
    with open('./JobList/'+str(i)+'.txt', 'w',encoding='utf-8') as f:
        f.write(df.loc[i,'Title']+ '\n')
        f.write(str(df.loc[i,'FullDescription']))

In [67]:
def coursemixture(mycourse):
    courselist=df_course.copy()
    courselist.drop(courselist.index, inplace=True)
    for i in mycourse:
        courselist=courselist.append(df_course[df_course["Code"] == i]  )
    return courselist

In [68]:
courselist=coursemixture(mycourse)

In [69]:
courselistintroduction=''
for i in courselist.index:
    courselistintroduction+=courselist.loc[i,'Course Introduction']

In [72]:
def clean_text(text):
    # text preprocessing
    REPLACE_BY_SPACE_RE = re.compile('[#+_/(){}!^?<>"''*\[\]\|@,;]')
    BAD_SYMBOLS_RE = re.compile('[^0-9a-zA-Z #+_]')
    match_regex = re.compile('\d+')
    STOPWORDS = set(stopwords.words('english'))
    
    text = re.sub(REPLACE_BY_SPACE_RE, ' ', text)  # replace REPLACE_BY_SPACE_RE symbols by space in text
    text = re.sub(BAD_SYMBOLS_RE, '', text)  # delete symbols which are in BAD_SYMBOLS_RE from text
    querywords = text.split()
    # change to lower-csae
    text = str(text).lower()
    # drop the stopwords
    text = ' '.join(word for word in text.split() if word not in STOPWORDS) 
    return text

### Calculate cosine similarity

In [74]:
def consine_similarity(course, job):

    doc = [course,job]
    cv = CountVectorizer()
    count_matrix = cv.fit_transform(doc)
    matchpercentage = cosine_similarity(count_matrix)[0][1]
    matchpercentage = round(matchpercentage,2)

    return(matchpercentage)

In [75]:
df_data['course match score']=''

In [76]:
for i in df_data.index:
    f = open('./JobList/'+str(i)+'.txt', 'r',encoding = 'utf -8')       
    jd = f.read()
    df_data.loc[i,'score']=consine_similarity(courselistintroduction,jd)


In [77]:
df_data = df_data.sort_values(by=['score'],ascending=False)
pd.reset_option('display.max_colwidth')
df_data.drop_duplicates(subset=['Title', 'FullDescription']).head(20)

,Unnamed: 0,Id,Title,FullDescription,LocationRaw,LocationNormalized,ContractType,ContractTime,Company,Category,SalaryRaw,SalaryNormalized,SourceName,FullDescription_Raw,course match score,score
2359,40652,68361191,Data Management Consultant,position data analyst data management consulta...,Wiltshire South West,Wiltshire,NaN,contract,Outsource UK Ltd,IT Jobs,350 - 390 per day,88800,cwjobs.co.uk,Position: Data Analyst/Data Management Consult...,,0.33
2947,49575,68627047,Data Management Consultant,position data analyst data management consulta...,Wiltshire Swindon SN256,Swindon,NaN,contract,Outsource UK,IT Jobs,350.00 - 390.00 GBP Day,88800,jobserve.com,Position: Data Analyst/Data Management Consult...,,0.33
3041,50027,68628626,Business Intelligence (BI) Consultants Advisory,business intelligence bi consultants advisory ...,London,London,NaN,permanent,Anson McCade Ltd,IT Jobs,35000.00 - 60000.00 GBP Annual,47500,jobserve.com,Business Intelligence (BI) Consultants Adviso...,,0.32
2379,40736,68361481,Senior Analyst EDS,business systems analyst glasgow salary harvey...,Glasgow Lanarkshire Scotland,UK,NaN,permanent,Harvey Nash plc,IT Jobs,25000 - 45000 per annum + Good benefits,35000,cwjobs.co.uk,"BUSINESS SYSTEMS ANALYST, Glasgow Salary **** ...",,0.32
4899,73713,68996464,Financial Analyst,cooper lomaz urgent requirement financial anal...,"Bury St Edmunds, Suffolk, England, Suffolk",Bury St. Edmunds,NaN,contract,Cooper Lomaz Bury St Edmunds,Accounting & Finance Jobs,250/day,60000,cv-library.co.uk,Cooper Lomaz has an urgent requirement for a F...,,0.31
3044,50048,68628693,"Business Analyst FINANCIAL SERVICES, Independ...",business analyst financial services independen...,London,London,NaN,permanent,Vantage Recruitment Solutions Limited,IT Jobs,40000.00 - 55000.00 GBP Annual,47500,jobserve.com,"Business Analyst FINANCIAL SERVICES, Independ...",,0.31
5228,79380,69018141,Technical Business Analyst,client looking business analyst financial serv...,London,London,NaN,permanent,Clearwater People Solutions,IT Jobs,45000 - 60000/annum,52500,cv-library.co.uk,Our client is looking for a Business Analyst w...,,0.31
3369,54551,68677427,Data Analyst,company data analyst investment management cit...,West London London South East,East Sheen,NaN,permanent,Robert Half Technology,IT Jobs,25000 - 30000 per annum + Excellent Package & ...,27500,totaljobs.com,The Company Data Analyst Investment Managemen...,,0.31
5955,88454,69140108,Business intelligence data programme manager Bank,business intelligence data programme manager e...,Scotland,Scotland,NaN,NaN,Huxley Associates,Other/General Jobs,"50,000-74,999 yearly",62499,theladders.co.uk,Business intelligence data programme manager E...,,0.31
7045,104894,69574530,Technical Business Analyst,client looking business analyst financial serv...,Midlothian Edinburgh EH120,Craigiehall,NaN,permanent,Clearwater People Solutions,IT Jobs,45000.00 - 60000.00 GBP Annual,52500,jobserve.com,Our client is looking for a Business Analyst w...,,0.31
